In [2]:
import numpy as np
import warnings
from quspin.operators import hamiltonian
from quspin.basis import spin_basis_1d
warnings.filterwarnings("ignore", category=FutureWarning)

[Libreria di riferimento](http://weinbe58.github.io/QuSpin/generated/quspin.basis.spin_basis_1d.html)

We want to write a chain of L coupled qubits:
$$H[h_x(t)] = - \sum_{j=1}^L(JS^z_{j+1}S^z_j+gS^z_j+h_x(t)S^x_j)$$
as in the article we set g=1. 

In the particoular case of single qubit we remove the first term J=0: 

$$H[h_x(t)] = - S^z_j - h_x(t)S^x_j$$

In [3]:
def hamiltonian0(L=1,J=1.0,g=1.0,hz=1.0, hx_min=-4., hx_max =4.,Δh=1,n_step=4): 
    if L < 2:
        J=0
    else:
        J=J
#     if L>1: 
    # Nel caso di una chain si fa riferimento a symmetries rappresentate da parametri block, capisci meglio.
    basis = spin_basis_1d(L=L,pauli=0,kblock=0,pblock=1) 
    # Voglio usare il il metodo hamiltoniana, come primo argomento gli serve 
    
    x_field=[[g,i] for i in range(L)]
    z_field=[[hz,i] for i in range(L)]
#     hx_discrete = np.arange(start=hx_min, stop=hx_max, step=Δh)
    # la parte dinamica dell' hamiltoniana nella funzione richiede una lista con
    # [["asse",coefficiente,funzione dipendente da t,parametri funzione dipendente da t]]
    J_nn=[[J,i,(i+1)%L] for i in range(L)] # Periodic Boundary Condition
    static=[["zz",J_nn],["z",z_field]]
    
    h_values =np.arange(hx_min,hx_max,Δh) # discrete set of possible h fields 
    
    def operator(time):
        return h_values[hx_discrete[int(time)]]
    
    hx_discrete = np.zeros(n_step, dtype=int) # hx_discrete are protocols specified as integers
    
    
#     fct = lambda time: h_set[hx_discrete[int(time)]]
    # If the operator is time-INdependent, one must pass an empty list: dynamic_list = [].
    dynamic=[["x",x_field,operator,[]]]
    H=hamiltonian(static,dynamic,static_fmt="dia",dtype=np.float64,basis=basis)
    print("\n\n-------------------------------> Hamiltonian <-------------------------------")
    print("")
    print(H.toarray())
    print("")
    print("------------------------------------------------------------------------------")

    print("\n\n---------------------> Eigenvalues, Eigenvectors (in columns)<-------------------------------")
    print("")
    print(H.eigh())
    print("")
    print("------------------------------------------------------------------------------")
    return H
hamiltonian0(L=3)

Hermiticity check passed!
Symmetry checks passed!


-------------------------------> Hamiltonian <-------------------------------

[[ 2.25       -3.46410162  0.          0.        ]
 [-3.46410162  0.25       -4.          0.        ]
 [ 0.         -4.         -0.75       -3.46410162]
 [ 0.          0.         -3.46410162 -0.75      ]]

------------------------------------------------------------------------------


---------------------> Eigenvalues, Eigenvectors (in columns)<-------------------------------

(array([-6.16876758, -1.61785763,  2.52563332,  6.26099189]), array([[ 0.22432566, -0.48462587,  0.64105626,  0.55123736],
       [ 0.54517616, -0.54111111, -0.05100788, -0.63826321],
       [ 0.68056804,  0.16701869, -0.5261522 ,  0.48176318],
       [ 0.43507252,  0.6666643 ,  0.55642513, -0.23803716]]))

------------------------------------------------------------------------------


<quspin.operators.hamiltonian:
static mat: <4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements (1 diagonals) in DIAgonal format>
dynamic:
 0) func: operator, mat: <4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format> >